# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
df = pd.read_csv("output_data/cities_weather.csv")
df.head()

,Unnamed: 0,City,Latitude,Longitude,Max temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hobart,-42.88,147.33,51.80,66,20,10.29,AU,1593397216
1,1,Kristinestad,62.27,21.38,61.75,86,67,9.60,FI,1593397216
2,2,Bluff,-46.60,168.33,48.65,81,96,14.85,NZ,1593397216
3,3,Rikitea,-23.12,-134.97,68.54,55,31,17.96,PF,1593397217
4,4,Lüderitz,-26.65,15.16,58.24,54,0,13.53,NaN,1593397217


In [12]:
df.count()

Unnamed: 0         591
City               591
Latitude           591
Longitude          591
Max temperature    591
Humidity           591
Cloudiness         591
Wind Speed         591
Country            588
Date               591
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

#Drop any NaN values from the dataset
clean_df = df.dropna()

In [14]:
# Store 'Lat' and 'Lng' into  locations 
locations = clean_df[["Latitude", "Longitude"]].astype(float)

#Store Humidity value 
humidity = clean_df["Humidity"].astype(float)

# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
#Locate the cities matching our conditions for ideal weather:
ideal_weather = clean_df.loc[(clean_df["Max temperature"]>=80) &
                             (clean_df["Max temperature"]<=95) &
                             (clean_df["Wind Speed"]<=10) &
                             (clean_df["Cloudiness"] == 0) ]
#Dropping unnecessary column                          
new_df = ideal_weather.drop(columns = ["Unnamed: 0"])
new_df

,City,Latitude,Longitude,Max temperature,Humidity,Cloudiness,Wind Speed,Country,Date
5,Bengkulu,-3.80,102.27,82.51,71,0,4.65,ID,1593397217
17,Arlit,18.74,7.39,87.44,13,0,6.15,NE,1593397222
31,Sibolga,1.74,98.78,85.33,61,0,0.96,ID,1593396934
68,Port Hedland,-20.32,118.57,82.40,18,0,4.70,AU,1593397242
81,Safaga,26.73,33.94,84.20,32,0,5.82,EG,1593397247
110,Sakakah,29.97,40.21,82.40,23,0,6.93,SA,1593397259
135,Dzhebariki-Khaya,62.22,135.80,88.72,29,0,3.11,RU,1593397269
146,Borogontsy,62.67,131.17,87.55,29,0,4.65,RU,1593397273
158,Churapcha,62.00,132.43,86.76,27,0,3.80,RU,1593397278
269,Esna,25.29,32.55,82.40,32,0,5.82,EG,1593397321


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
#Store Latitude, Longitude, City and Country into Hotel_df
hotel_df = new_df[["Latitude","Longitude","City","Country"]]
#Add extra column Hotel Name
hotel_df["Hotel Name"]=''
hotel_df

C:\Users\Babette\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Latitude,Longitude,City,Country,Hotel Name
5,-3.80,102.27,Bengkulu,ID,
17,18.74,7.39,Arlit,NE,
31,1.74,98.78,Sibolga,ID,
68,-20.32,118.57,Port Hedland,AU,
81,26.73,33.94,Safaga,EG,
110,29.97,40.21,Sakakah,SA,
135,62.22,135.80,Dzhebariki-Khaya,RU,
146,62.67,131.17,Borogontsy,RU,
158,62.00,132.43,Churapcha,RU,
269,25.29,32.55,Esna,EG,


In [18]:
#Set parameters to search for hotels with 5000 meters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_search = "hotel"
target_radius = 5000

# set up a parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get target coordinates type from new_df
    target_coordinates = (f"{row['Latitude']}, {row['Longitude']}")
    
    # add target coordinates to params dict
    params['location'] = target_coordinates

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:     
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
               
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index, 'Hotel Name'] = "NA"

C:\Users\Babette\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [20]:
hotel_df

,Latitude,Longitude,City,Country,Hotel Name
5,-3.80,102.27,Bengkulu,ID,Hotel Santika Bengkulu
17,18.74,7.39,Arlit,NE,NA
31,1.74,98.78,Sibolga,ID,Prima Indah Hotel
68,-20.32,118.57,Port Hedland,AU,The Esplanade Hotel
81,26.73,33.94,Safaga,EG,Lamar Resort Abu Soma
110,29.97,40.21,Sakakah,SA,Raoum Inn Hotel
135,62.22,135.80,Dzhebariki-Khaya,RU,NA
146,62.67,131.17,Borogontsy,RU,NA
158,62.00,132.43,Churapcha,RU,NA
269,25.29,32.55,Esna,EG,Orient Star Dahabyia Isna 05 Nights Each Sunday


In [21]:
#As we can't find any hotel in a few cities we will drop the NA values
df = hotel_df.loc[hotel_df["Hotel Name"] != 'NA']
df

,Latitude,Longitude,City,Country,Hotel Name
5,-3.80,102.27,Bengkulu,ID,Hotel Santika Bengkulu
31,1.74,98.78,Sibolga,ID,Prima Indah Hotel
68,-20.32,118.57,Port Hedland,AU,The Esplanade Hotel
81,26.73,33.94,Safaga,EG,Lamar Resort Abu Soma
110,29.97,40.21,Sakakah,SA,Raoum Inn Hotel
269,25.29,32.55,Esna,EG,Orient Star Dahabyia Isna 05 Nights Each Sunday
321,38.47,106.27,Yinchuan,CN,Kempinski Hotel Yinchuan


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in df.iterrows()]
locations = df[["Latitude", "Longitude"]]

In [24]:
# Add marker layer ontop of heat map

# Create hotel layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5,
    info_box_content=hotel_info
)



fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))